In [2]:
import numpy as np
import pandas as pd

In [4]:
breakdowns = pd.read_csv('desolationofeden_guitarkick.csv')

In [5]:
breakdowns.drop(breakdowns.columns[[0, 1]], axis=1, inplace=True)

In [129]:
breakdowns.head(10)

,Kick,E,A,Articulation
0,0.0,1,0.0,0.0
1,0.0,1,0.0,0.0
2,0.0,1,0.0,0.0
3,0.0,1,0.0,1.0
4,NaN,NaN,NaN,NaN
5,0.0,1,0.0,0.0
6,0.0,1,0.0,1.0
7,NaN,NaN,NaN,NaN
8,0.0,1,0.0,0.0
9,0.0,1,0.0,0.0


In [6]:
X = breakdowns.fillna(-1)

### Get "Vocab" (number of unique row configurations)

In [8]:
def get_vocab(X):
    full_vocab = []
    vocab = []
    indices_vocab = dict()
    vocab_indices = dict()
    for row in X.as_matrix():
        full_vocab.append(list(row))
        if list(row) not in vocab:
            vocab.append(list(row))
            indices_vocab.update({vocab.index(list(row)): list(row)})
            vocab_indices.update({str(list(row)): vocab.index(list(row))})
    return full_vocab, vocab, indices_vocab, vocab_indices

In [9]:
full_vocab, vocab, indices_vocab, vocab_indices = get_vocab(X)

In [137]:
vocab_indices

{"[-1.0, '0', -1.0, 3.0]": 5,
 "[-1.0, '0', 0.0, 0.0]": 13,
 "[-1.0, '1', -1.0, 2.0]": 38,
 "[-1.0, '1', 0.0, 0.0]": 45,
 "[-1.0, '2', -1.0, 3.0]": 27,
 "[-1.0, '3', -1.0, 3.0]": 26,
 "[-1.0, '3', -1.0, 4.0]": 14,
 "[-1.0, '6', -1.0, 3.0]": 25,
 '[-1.0, -1, -1.0, -1.0]': 2,
 "[0.0, '0', -1.0, 2.0]": 6,
 "[0.0, '0', -1.0, 3.0]": 3,
 "[0.0, '0', 0.0, 0.0]": 7,
 "[0.0, '0', 0.0, 1.0]": 34,
 "[0.0, '0', 0.0, 3.0]": 9,
 "[0.0, '0', 1.0, 0.0]": 22,
 "[0.0, '1', -1.0, 2.0]": 37,
 "[0.0, '1', 0.0, 0.0]": 0,
 "[0.0, '1', 0.0, 1.0]": 1,
 "[0.0, '1', 1.0, 0.0]": 19,
 "[0.0, '1', 1.0, 3.0]": 8,
 "[0.0, '11', -1.0, 3.0]": 43,
 "[0.0, '2', -1.0, 2.0]": 36,
 "[0.0, '2', -1.0, 3.0]": 23,
 "[0.0, '2', 1.0, 0.0]": 32,
 "[0.0, '2', 2.0, 1.0]": 41,
 "[0.0, '3', -1.0, 2.0]": 35,
 "[0.0, '3', -1.0, 3.0]": 16,
 "[0.0, '3', -1.0, 4.0]": 15,
 "[0.0, '3', 2.0, 0.0]": 31,
 "[0.0, '3', 3.0, 0.0]": 21,
 "[0.0, '3', 3.0, 1.0]": 40,
 "[0.0, '3', 3.0, 3.0]": 10,
 "[0.0, '4', 4.0, 0.0]": 20,
 "[0.0, '6', -1.0, 3.0]": 

### Split into Semi-Redundant Sequences of Fixed Length

In [10]:
maxlen = 32
step = 4
sequences = []
next_beat = []

In [11]:
for i in range(0, len(full_vocab)-maxlen, step):
    sequences.append(full_vocab[i: i+maxlen])
    next_beat.append(full_vocab[i + maxlen])

##### Helper Function from fchollet/keras/blob/master/examples/lstm_text_generation.py

In [12]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### Vectorization

In [13]:
X = np.zeros((len(sequences), maxlen, len(vocab)), dtype=np.bool)
y = np.zeros((len(sequences), len(vocab)), dtype=np.bool)

In [14]:
for i, measures in enumerate(sequences):
    for t, beat in enumerate(measures):
        X[i, t, vocab_indices[str(beat)]] = 1
    y[i, vocab_indices[str(next_beat[i])]] = 1

In [131]:
X.shape

(924, 32, 47)

In [139]:
X[0][3]

array([False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False], dtype=bool)

In [135]:
X[0]

array([[ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ..., 
       [ True, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False]], dtype=bool)

### Model

In [15]:
import sys
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [125]:
model = Sequential()

model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(len(vocab)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [121]:
from keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

In [123]:
tensorboard.set_model(model)

### Train Model

In [ ]:
for iteration in range(1, 201):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1,
              callbacks=[tensorboard])

    start_index = np.random.randint(0, (len(full_vocab)-maxlen)//32)
    start_index = start_index * 32
    
    if iteration%50 == 0 or iteration == 1:
        
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print('----- diversity:', diversity)

            generated = []
            sequence = full_vocab[start_index: start_index + maxlen]
            generated.append(sequence)
            
            print('===== seed:', sequence)
        
            for i in range(256):
                x = np.zeros((1, maxlen, len(vocab)))
                for t, beat in enumerate(sequence):
                    x[0, t, vocab_indices[str(beat)]] = 1.

                preds = model.predict(x, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_beat = indices_vocab[next_index]

                generated.append(next_beat)
                sequence = sequence[1:]
                sequence.append(next_beat)
            
                sys.stdout.write(str(next_beat)+'\n')
                sys.stdout.flush()
            print()   